In [ ]:
! pip install polars

In [ ]:
import polars as pl
import datetime as dt

df = pl.DataFrame(
    {
        "name": ["Alice Archer", "Ben Brown", "Chloe Cooper", "Daniel Donovan"],
        "birthdate": [
            dt.date(1997, 1, 10),
            dt.date(1985, 2, 15),
            dt.date(1983, 3, 22),
            dt.date(1981, 4, 30),
        ],
        "weight": [57.9, 72.5, 53.6, 83.1],  # (kg)
        "height": [1.56, 1.77, 1.65, 1.75],  # (m)
    }
)

print(df)

Lazy Loading of the dataset

In [ ]:
df = pl.scan_parquet("https://nam04.safelinks.protection.outlook.com/?url=https%3A%2F%2Fdata.ljrobins.com%2Ffiles%2Fpage.parquet&data=05%7C02%7Cbkhwaja%40purdue.edu%7C37993b20219243afa89c08ddc561d36f%7C4130bd397c53419cb1e58758d6d63f21%7C1%7C0%7C638883746144100445%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=FaRLG%2F0XFctogRNcYmjtH3lE6D7cNlry7fojrlA0dos%3D&reserved=0")
# print(df)
print(df.explain())
links = pl.scan_parquet("https://nam04.safelinks.protection.outlook.com/?url=https%3A%2F%2Fdata.ljrobins.com%2Ffiles%2Flinkpairs.parquet&data=05%7C02%7Cbkhwaja%40purdue.edu%7C37993b20219243afa89c08ddc561d36f%7C4130bd397c53419cb1e58758d6d63f21%7C1%7C0%7C638883746144130439%7CUnknown%7CTWFpbGZsb3d8eyJFbXB0eU1hcGkiOnRydWUsIlYiOiIwLjAuMDAwMCIsIlAiOiJXaW4zMiIsIkFOIjoiTWFpbCIsIldUIjoyfQ%3D%3D%7C0%7C%7C%7C&sdata=Uk3kXId438qv6o%2FN2Ax4zYlPC00mW6ZD2KXR6qfc9bU%3D&reserved=0")
print(links.explain())

In [ ]:
page = df.collect()
linkPairs = links.collect()

Graph Creation

In [ ]:
print(page)
print(linkPairs)
# edge definition -> linkPairs
# node definition -> page_title (get from pages)

class GraphNode:

  def __init__(self, nodeId=0, nodeName="") -> None:
    self.nodeId = nodeId
    self.nodeName = nodeName
    self.edges = []

  def addEdge(self, otherId) -> None:
    self.edges.append(otherId)

  def __str__(self) -> str:
      return f"Graph node {self.nodeId} with the name {self.nodeName} has these connection {self.edges}"


In [ ]:
# create graph dictionary
graph = {}
for row in page.iter_rows():
  id, name = row
  node = GraphNode(nodeId=id, nodeName=name)
  graph.update({id : node})

# create edges
for row in linkPairs.iter_rows():
  plFrom, plTo = row
  node = graph[plFrom]
  node.addEdge(otherId=plTo)


In [ ]:
# del page
# del linkPairs

import gc
gc.collect()

### Question :

How to effectively create study plan for students using Wikipedia?

Further Questions:
Finding other people that are reseraching the same topics as you

Create an Example User Class

In [ ]:
import numpy as np

class User:

  def __init__(self, name="") -> None:
    self.name = name
    self.visitedPages = {} # holds the ids visited and its frequency of times visited
    self.recommendedPages = [] # id of the wiki pages

  def addPage(self, id) -> None:
    if id in self.visitedPages:
      self.visitedPages[id] += 1
    else:
      self.visitedPages.update({id : 1})

  def addRecommendedPapges(self, id) -> None:
    self.recommendedPages.append(id)

  def __str__(self) -> str:
    return f"{self.name} has visited these pages {self.visitedPages}"




In [ ]:
# Create a recommendation algorithm for the pages visited by the user
# possible articles in the vicinity of the pages visited by the user -> BFS
user = User(name="Basil Khwaja")
random_idx = np.random.randint(0, len(graph), size=10)
id_keys = list(graph.keys())
for idx in random_idx:
  user.addPage(id_keys[idx])

# print(user)

def recommendArticles(user):

  visitedPages = user.visitedPages
  visitedKeys = list(user.visitedPages.keys())
  print(f"Pages visited by {user.name}")
  for key in visitedKeys:
    node = graph.get(key)
    print(node.nodeName)

  print("\n")

  cluster = {}
  maxDistance = 2 # changes to see different recommendation results

  def bfs(start_id, max_distance) -> None:
      queue = [(start_id, 0)]  # (node_id, distance)

      while queue:
          current_id, distance = queue.pop(0)

          # if current_id not in visitedKeys:
          if current_id in cluster:
            cluster[current_id] += 1
          else:
            cluster.update({current_id : 1})

          if distance < max_distance:
              node = graph.get(current_id)
              if node:
                  for neighbor in node.edges:
                      queue.append((neighbor, distance + 1))

  for key, value in visitedPages.items():
    bfs(key, maxDistance)

  # print(cluster)

  # overlap in graph is more of importance that means there is a shared article of importance
  # sort by value of descencing overlap in graph
  clusterDesc = dict(sorted(cluster.items(), key=lambda item: item[1], reverse=True))
  # print(clusterDesc)

  # Recommend the user these articles to read
  recommendList = list(clusterDesc.keys())
  # print(f"Recommend {user.name} these next articles to read : ")
  # for i in range(5):
  #   node = graph.get(recommendList[i])
  #   print(node.nodeName)

  return recommendList

recommendList = recommendArticles(user)

In [ ]:
!pip install Wikipedia-API

In [ ]:
import wikipediaapi

def get_wiki_summary(articleTitle):
    wiki = wikipediaapi.Wikipedia(user_agent='StudyPlan (khwajabasil@gmail.com)', language='en')
    page = wiki.page(articleTitle)

    if page.exists():
        return page.summary
    else:
        return f"Could not find a summary for '{articleTitle}' on Wikipedia."

recommendedTitles = []
for i in range(5):
  node = graph.get(recommendList[i])
  recommendedTitles.append(node.nodeName)

print("Summary of Recommended Articles to look into")
print("="*40)
for recommended in recommendedTitles:
  summary = get_wiki_summary(recommended)
  print(f"\n--- {recommended} ---")
  print(summary)
  print("-" * 20)

In [ ]:
# !pip install nx-cugraph-cu11 --extra-index-url https://pypi.nvidia.com
!nvidia-smi
# !pip install nx-cugraph-cu12 --extra-index-url https://pypi.anaconda.org/rapidsai-wheels-nightly/simple
# !pip uninstall cupy-cuda11x cupy-cuda12x
! pip install cupy-cuda12x

### Grab the Subgraph of the User

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import cupy
print(cupy.__version__)
import nx_cugraph as nxcg


# Create a new directed graph for the subgraph
subgraph = nx.Graph()

maxDistance = 1
visitedPages = user.visitedPages

for pageId in visitedPages.keys():
    if pageId in graph:
        node = graph[pageId]
        subgraph.add_node(node.nodeId, name=node.nodeName, label=node.nodeName, type='visited')

nodesToExplore = list(visitedPages.keys())
visitedInBfs = set(visitedPages.keys())

for start_node_id in nodesToExplore:
    queue = [(start_node_id, 0)]

    while queue:
        current_id, distance = queue.pop(0)

        if current_id in graph:
            current_node = graph[current_id]

            if current_id not in subgraph.nodes():
                 subgraph.add_node(current_node.nodeId, name=current_node.nodeName, label=current_node.nodeName, type='neighbor')

            if distance < maxDistance:
                for neighbor_id in current_node.edges:
                    if neighbor_id in graph:
                         neighbor_node = graph[neighbor_id]
                         if neighbor_id not in subgraph.nodes():
                              subgraph.add_node(neighbor_node.nodeId, name=neighbor_node.nodeName, label=neighbor_node.nodeName, type='neighbor')

                         subgraph.add_edge(current_id, neighbor_id)

                         if neighbor_id not in visitedInBfs:
                             visitedInBfs.add(neighbor_id)
                             queue.append((neighbor_id, distance + 1))


print(f"Created a subgraph with {subgraph.number_of_nodes()} nodes and {subgraph.number_of_edges()} edges.")

# convert to cuGraph for better GPU optimization becuase my CPU is cooked :(

nxcgSubgraph = nxcg.from_networkx(subgraph)
fq2Layout = nxcg.forceatlas2_layout(nxcgSubgraph, max_iter=500)
# Fix: Iterate through the dictionary items directly
pos = {int(node_id): (float(coords[0]), float(coords[1])) for node_id, coords in fq2Layout.items()}

# Scale out the plot by increasing figure size
plt.figure(figsize=(20, 20)) # You can adjust the size as needed

nx.draw(subgraph, pos=pos, with_labels=True, node_size=300, font_size=10)
plt.show()

In [ ]:
import cupy
print(cupy.__version__)
import nx_cugraph as nxcg

print("GPU available:", cupy.cuda.is_available())
nxcgSubgraph = nxcg.from_networkx(subgraph)
fq2Layout = nxcg.forceatlas2_layout(nxcgSubgraph, max_iter=500)
pos = {int(node_id): (float(coords[0]), float(coords[1])) for node_id, coords in fq2Layout.items()}
nx.draw(subgraph, pos=pos, with_labels=True, node_size=20, font_size=5)
plt.figure(figsize=(100,100))
plt.show()

In [ ]:
! pip install pyvis

OMG NEVER LET THE GRAPH GET OUT OF CONTROL
---
Changing the max distance from 2 -> 1 helped render the graph SO much better
---
At most rn form clusters of one because of the amount of connections that exist between the wikepedia pages

In [ ]:
from pyvis.network import Network
from google.colab import files

nt = Network(height="750px", width="70%", bgcolor="#222222", font_color="white")
nt.from_nx(subgraph)
# nt.toggle_physics(True)
nt.show_buttons(filter_=['physics'])
nt.save_graph("subgraph.html")
# files.download("graph.html")
# nt.show("graph.html")

### Simulate Multiple Users

Learn about the user more

In [ ]:
class Platform:

  def __init__(self) -> None:
     self.users = []

  def addUser(self,user):
    self.users.append(user)

  def __str__(self) -> str:
     return f"{len(self.users)} Users exist on the platform"

In [ ]:
platform = Platform()
numberOfUsers = 5
for i in range(0, numberOfUsers):
  user = User(f"User {i}")
  platform.addUser(user)

print(platform)

for user in platform.users:
  random_idx = np.random.randint(0, len(graph), size=10)
  id_keys = list(graph.keys())
  for idx in random_idx:
    user.addPage(id_keys[idx])

  recommnedations = recommendArticles(user)
  # print(recommnedations)

  user.recommendedPages = recommnedations

Embeddings were cooking my RAM :(

We can more experiments on the data given more time and better resources. My poor collab about to start crying

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

usersRecommendedTitles = {user.name: user.recommendedPages for user in platform.users}

# Load a pre-trained sentence transformer model
# Ensure the model is on the GPU if available
model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')

userNames = list(usersRecommendedTitles.keys())
nums = len(userNames)
similarityScores = {}

def convertIdtoNames(idList):
  newTitles = []
  for id in idList:
    node = graph.get(id)
    newTitles.append(node.nodeName)

  return newTitles

for i in range(len(userNames)):
  for j in range(i + 1, len(userNames)):
    user1 = userNames[i]
    user2 = userNames[j]

    id1 = usersRecommendedTitles[user1]
    id2 = usersRecommendedTitles[user2]

    titles1 = convertIdtoNames(id1)
    titles2 = convertIdtoNames(id2)

    if not titles1 or not titles2:
        continue  # Skip if either user has no valid titles

    # Encode titles into embeddings (NO precision arg)
    embeddings = model.encode(titles1, convert_to_numpy=True)
    avg = np.mean(embeddings, axis=0)
    normalized_avg1 = avg / np.linalg.norm(avg)
    embeddings2 = model.encode(titles2, convert_to_numpy=True)
    avg = np.mean(embeddings2, axis=0)
    normalized_avg2 = avg / np.linalg.norm(avg)

    similarity = np.dot(normalized_avg1, normalized_avg2)

    similarityScores[(user1, user2)] = similarity

print("Semantic Similarity Scores Between Users:")
for (user1, user2), score in list(similarityScores.items())[:10]:
  print(f"{user1} and {user2}: {score:.4f}")

In [ ]:
! pip install datasketch

In [ ]:
from datasketch import MinHash
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def get_minhash(titles):
    m = MinHash(num_perm=128)
    for title in titles:
        for word in title.lower().split():
            m.update(word.encode('utf8'))
    return m

minhashes = {user.name: get_minhash(convertIdtoNames(usersRecommendedTitles[user.name])) for user in platform.users}

jaccardSimilarityScores = {}
userNames = list(minhashes.keys())
n = len(userNames)

for i in range(n):
    for j in range(i + 1, n):
        user1 = userNames[i]
        user2 = userNames[j]

        minhash1 = minhashes[user1]
        minhash2 = minhashes[user2]

        sim = minhash1.jaccard(minhash2)

        jaccardSimilarityScores[(user1, user2)] = sim

print("\nMinHash Jaccard Similarity Scores Between Users:")
for (user1, user2), score in jaccardSimilarityScores.items():
    print(f"{user1} and {user2}: {score:.4f}")

heatmap_data = pd.DataFrame(0.0, index=userNames, columns=userNames)

for (user1, user2), score in jaccardSimilarityScores.items():
    heatmap_data.loc[user1, user2] = score
    heatmap_data.loc[user2, user1] = score

plt.figure(figsize=(10, 8)) # Adjust figure size as needed
sns.heatmap(heatmap_data, annot=True, cmap='Blues', fmt=".2f", linewidths=.5)
plt.title('MinHash Jaccard Similarity Heatmap Between Users')
plt.xlabel('User')
plt.ylabel('User')
plt.show()

In [ ]:
import gc
gc.collect()